# 3. Transition Analysis

Analyze and visualize cohort-level state transitions.

## Contents
1. Transition Matrix Estimation
2. Visualization
3. Transition Transformer Training
4. Cohort-Conditional Transitions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from privatecredit.data import LoanTapeGenerator
from privatecredit.calibration import TransitionCalibrator
from privatecredit.models.baseline_markov import MarkovTransitionModel

## 1. Transition Matrix Estimation

In [ ]:
# Credit states
STATES = ['Performing', '30DPD', '60DPD', '90DPD', 'Default', 'Prepaid', 'Matured']

# Calibrate to target rates
calibrator = TransitionCalibrator()
P = calibrator.fit_to_target_rates(
    target_default_rate=0.02,
    target_prepayment_rate=0.15,
    maturity=60
)

print("Calibrated Transition Matrix:")
print(pd.DataFrame(P, index=STATES, columns=STATES).round(4))

## 2. Visualization

In [ ]:
# Heatmap of transition probabilities
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(P, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=STATES, yticklabels=STATES, ax=ax)
ax.set_title('State Transition Probabilities (Monthly)')
ax.set_xlabel('To State')
ax.set_ylabel('From State')
plt.tight_layout()
plt.show()

In [ ]:
# Cumulative state probabilities over time
n_periods = 60
state_probs = np.zeros((n_periods, 7))
state_probs[0, 0] = 1.0  # Start in Performing

for t in range(1, n_periods):
    state_probs[t] = state_probs[t-1] @ P

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
for i, state in enumerate(STATES):
    ax.plot(state_probs[:, i], label=state, linewidth=2)

ax.set_xlabel('Month')
ax.set_ylabel('Probability')
ax.set_title('State Distribution Over Time')
ax.legend(loc='center right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Transition Transformer Training

In [ ]:
from privatecredit.models import TransitionTransformer
from privatecredit.models.transition_transformer import TransitionTransformerConfig

# Configuration
config = TransitionTransformerConfig(
    n_states=7,
    n_macro_vars=9,
    hidden_dim=128,
    n_heads=4,
    n_layers=3
)

model = TransitionTransformer(config)
print(f"Transition Transformer parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Cohort-Conditional Transitions

Different cohorts (asset classes, vintages) have different transition dynamics.

In [ ]:
# Baseline default rates by asset class
asset_class_defaults = {
    'corporate': 0.02,
    'consumer': 0.04,
    'real_estate': 0.015,
    'receivables': 0.03
}

# Calibrate matrices for each
cohort_matrices = {}
for asset_class, default_rate in asset_class_defaults.items():
    P_cohort = calibrator.fit_to_target_rates(
        target_default_rate=default_rate,
        target_prepayment_rate=0.12
    )
    cohort_matrices[asset_class] = P_cohort

# Compare default paths
fig, ax = plt.subplots(figsize=(10, 6))
for name, P_c in cohort_matrices.items():
    probs = np.zeros(60)
    state = np.zeros(7)
    state[0] = 1.0
    for t in range(60):
        state = state @ P_c
        probs[t] = state[4]  # Default state
    ax.plot(probs, label=name.title(), linewidth=2)

ax.set_xlabel('Month')
ax.set_ylabel('Cumulative Default Probability')
ax.set_title('Default Curves by Asset Class')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

- Transition matrices capture monthly state evolution
- Absorbing states: Default, Prepaid, Matured
- Cohort-specific dynamics via calibration

**Next:** Loan trajectory simulation (Notebook 04)